# View data

In [ ]:
import json

import ipywidgets as widgets
from IPython.display import display, Markdown

import xml.etree.ElementTree as ET

from pathlib import Path

from tqdm import tqdm

from typing import Any

from utils import *

In [ ]:
query_id_pairs = []
with open(QUERY2ID_LIST_FPATH, "r") as f:
    for line in f:
        for query, id in json.loads(line).items():
            query_id_pairs.append((query, id))

print(f"# of query-id pairs: {len(query_id_pairs)}")
all_uniq_ids = set([id for _, id in query_id_pairs])
print(f"# of unique ids: {len(all_uniq_ids)}")

# of query-id pairs: 1743
# of unique ids: 997


In [ ]:
from transformers import AutoTokenizer

model_id = "infgrad/stella-base-zh-v3-1792d"

TEST_STR = """广东省深圳市中级人民法院 民事判决书 （2010）深中法民六终字第5494号 上诉人（原审被告）中国（深圳）×企业股份有限公司。 法定代表人高某某，董事长。 委托代理人屈某某，男。 被上诉人（原审原告）李某某，男。 委托代理人田某某，江苏维×律师事务所深圳分所律师。 上诉人中国（深圳）×企业股份有限公司（以下简称中国×公司）与被上诉人李某某拖欠工资及经济补偿金纠纷一案，中国×公司不服深圳市福田区人民法院（2009）深福法民四初字第3698号民事判决，向本院提起上诉。本院依法组成合议庭审理了本案，现已审理终结。 原审判决查明，上诉人中国×公司董事会于2000年5月26日聘任被上诉人李某某为常务副总经理，主持公司经营和管理公司全面的工作。双方未签订劳动合同。2004年2月，经上诉人中国×公司董事会决议，聘任被上诉人李某某为董事长兼总经理。2005年3月，上诉人中国×公司经改制变更为中外合资企业，并召开了改制后的第一次股东大会。被上诉人李某某被选为董事会董事，并被聘任为董事长。2006年8月，上诉人中国×公司召开2006年度第二次董事会会议，选举李健为董事长，将被上诉人李某某改聘为副董事长。 被上诉人李某某主张其工作年限自1988年6月入职中国（深圳）×企业总公司（后更名为上诉人中国×公司）。后经上诉人中国×公司调任，先后在中国×服务中心（深圳）企业开发公司、深圳新高特实业公司（后更名为深圳市新特高实业有限公司）任职。被上诉人李某某认为，其在此期间的工龄应当连续计算。被上诉人李某某提供了上诉人中国×公司工商登记资料，该资料显示1998年5月中国（深圳）×企业总公司更名为上诉人中国×公司。上诉人中国×公司系中国×服务中心（深圳）企业开发公司、深圳新特高实业有限公司股东。 2006年9月30日起，上诉人中国×公司以被上诉人李某某管理不善为由要求被上诉人李某某停止工作。2006年10月11日，上诉人中国×公司召开2006年度第三次董事会会议，以被上诉人李某某不能完成公司的经营目标，造成巨额经营亏损为由，免除了被上诉人李某某的总经理职务。2006年10月24日，上诉人中国×公司再次召开董事会会议，以被上诉人李某某不再适应担任副董事长职位为由，免除了被上诉人李某某的副董事长职务。被上诉人李某某在职期间每月应发工资为10120元，实发工资为8375.56元。2006年6月、7月、8月、9月及11月，上诉人中国×公司每月发放被上诉人李某某工资1000元；2006年10月工资已按8375.56元的标准全额发放。2006年12月起，上诉人中国×公司未再支付被上诉人李某某工资。 2007年2月1日，被上诉人李某某向深圳市劳动争议仲裁委员会提起申诉，请求：1、确认双方的劳动关系依然存续；2、上诉人中国×公司支付被上诉人李某某2006年6月至2007年1月的工资66840元及25％的经济补偿金16710元；3、上诉人中国×公司承担全部仲裁费用。深圳市劳动争议仲裁委员会于2007年4月2日作出深劳仲案（2007）119号仲裁裁决，裁决如下：一、被上诉人李某某与上诉人中国×公司的事实劳动关系在2006年12月8日后依然延续；二、上诉人中国×公司支付被上诉人李某某2006年6月至2007年1月期间的工资差额54324.14元及25％的经济补偿金13581.04元；三、驳回被上诉人李某某的其他申诉请求。 上诉人中国×公司不服该裁决，于2007年4月29日向深圳市福田区人民法院提起诉讼。深圳市福田区人民法院于2007年7月4日作出（2007）深福法民一（劳）初字第799号民事判决，判决结果与仲裁裁决一致。上诉人中国×公司不服该判决，上诉至深圳市中级人民法院。深圳市中级人民法院于2008年4月2日作出（2007）深中法民六终字第4261号民事裁定，裁定撤销（2007）深福法民一（劳）初字第799号民事判决，发回重审。深圳市福田区人民法院于2009年5月31日作出（2008）深福法民四重字第1号民事判决，认为："上诉人中国×公司以被上诉人李某某不能完成公司经营指标，造成巨额经营亏损为由，免去被上诉人李某某公司董事长、副董事长、总经理职务并解除其与上诉人中国×公司之间的事实劳动关系。被上诉人李某某要求确认双方之间仍然存在劳动关系的主张，因上诉人中国×公司不同意，且双方关于继续履行事实劳动关系的具体内容及期限无法协商达成一致，如果双方事实劳动关系仍然延续，将不利于上诉人中国×公司的经营和稳定，也不利于被上诉人李某某事业的发展。因此，本院认为应当认定被上诉人李某某与上诉人中国×公司双方的劳动关系已经解除。……判决如下：一、被上诉人李某某与上诉人中国×公司之间的事实劳动关系已解除；二、上诉人中国×公司应自本判决生效之日起十日内支付被上诉人李某某2006年6月至2007年1月期间的工资差额54324.14元"。2009年6月11日被上诉人李某某与上诉人中国×公司收到该判决书后均未在法定期限内提出上诉。上诉人中国×公司依（2008）深福法民四重字第1号民事判决第二项内容支付了被上诉人李某某2006年6月至2007年1月期间的工资差额。 2009年6月26日被上诉人李某某再次向深圳市劳动争议仲裁委员会提出申诉，请求：1、被上诉人李某某与上诉人中国×公司事实劳动关系于2008年6月18日解除；2、上诉人中国×公司补发2006年1月至2008年6月期间的工资182160元，教育部机关服务中心承担连带责任；3、上诉人中国×公司支付拖欠工资的经济补偿金45540元，教育部机关服务中心承担补充责任；4、上诉人中国×公司支付解除劳动关系经济补偿金212520元及额外经济补偿金129030元，教育部机关服务中心承担补充责任；5、上诉人中国×公司支付任职期间（2000年1月至2008年6月）报销费用等126797.76元，支付拖欠的奖励380000元，教育部机关服务中心承担共同责任。深圳市劳动争议仲裁委员会作出深劳仲案（2009）2529号仲裁裁决，裁决如下：1、上诉人中国×公司支付被上诉人李某某解除劳动关系经济补偿金70840元和额外经济补偿金35420元；2、驳回被上诉人李某某的其他仲裁请求。 原审判决认为，被上诉人李某某与上诉人中国×公司虽未签订劳动合同，但上诉人中国×公司通过法定程序先后聘任被上诉人李某某为董事长、副董事长及总经理，被上诉人李某某也实际参与了上诉人中国×公司的经营管理，上诉人中国×公司根据被上诉人李某某提供的劳动向其支付了相应的工资报酬，可以认定被上诉人李某某与上诉人中国×公司之间已形成了事实劳动关系，双方的合法权益均应受我国劳动法保护。关于被上诉人李某某与上诉人中国×公司双方解除劳动关系的时间，被上诉人李某某在2007年2月1日劳动仲裁中要求确认双方劳动关系存续，并未提出解除双方劳动关系。仲裁{指深劳仲案（2007）119号仲裁裁决}支持了被上诉人李某某的请求。上诉人中国×公司不服仲裁{指深劳仲案（2007）119号仲裁裁决}而起诉。一审驳回了上诉人中国×公司的请求，上诉人中国×公司上诉而发回重审，本院于2009年5月31日重审判决双方劳动关系解除。因此，双方劳动关系在此期间并未解除。现被上诉人李某某主张双方劳动关系于2008年6月18日解除，系在此期间内，结合此期间上诉人中国×公司工商登记的法定代表人一直系被上诉人李某某，据此本院认定双方劳动关系于2008年6月18日解除。因此上诉人中国×公司应当支付被上诉人李某某2007年2月至2008年6月18日期间的工资。此期间被上诉人李某某处于停工，故上诉人中国×公司应支付被上诉人李某某此期间的工资103200元（10120×60％×16＋10120÷21.75×13），上诉人中国×公司未及时支付的，还应支付25％的经济补偿金25800元，被上诉人李某某要求超出部分的请求，没有法律依据，本院不予支持。关于被上诉人李某某的解除劳动关系经济补偿金，被上诉人李某某于1988年6月入职了上诉人中国×公司前身中国（深圳）×企业总公司，后经上诉人中国×公司调任，先后任职的中国×服务中心（深圳）企业开发公司、深圳新高特实业公司（后更名为深圳市新特高实业有限公司），虽然是独立法人资格，但上诉人中国×公司均系该两公司的股东，因此可以认定该两公司系上诉人中国×公司的子公司。被上诉人李某某系在上诉人中国×公司母子公司之间内部调动，被上诉人李某某的工龄可以连续计算。以解除劳动关系前被上诉人李某某正常工作的月平均工资为计算基数，上诉人中国×公司应支付被上诉人李某某解除劳动关系经济补偿金202400元（10120×20），上诉人中国×公司未按规定支付被上诉人李某某解除劳动关系经济补偿金的，还应支付50％的额外经济补偿金101200元，被上诉人李某某要求超出部分的请求，没有事实依据，本院不予支持。关于支付报销费用的诉请，被上诉人李某某要求上诉人中国×公司支付其任职期间的报销费用，该请求不属于劳动争议仲裁处理的范围，被上诉人李某某应另寻其他法律途径解决。关于支付各项奖励金的诉请，被上诉人李某某提供的证据材料均为复印件，无法单独作为认定事实的依据，上诉人中国×公司也不确认该证据，被上诉人李某某要求上诉人中国×公司支付各项奖励金的请求，证据不足，本院不予支持。综上所述，根据《深圳经济特区劳动合同条例》第三十一条、《违反和解除劳动合同的经济补偿办法》第十条、《深圳市员工工资支付条例》第五条、第十二条、第五十八条、第六十三条，《深圳经济特区企业工资管理暂行规定》第五十四条，《中华人民共和国民事诉讼法》第六十四条第一款、第一百二十八条之规定，判决如下：一、被上诉人李某某与上诉人中国×公司的事实劳动关系于2008年6月18日解除；二、上诉人中国×公司应支付被上诉人李某某2007年2月至2008年6月18日期间的工资103200元及25％的经济补偿金25800元；三、上诉人中国×公司应支付被上诉人李某某解除劳动关系的经济补偿金202400元及50％的额外经济补偿金101200元；上述款项，上诉人中国×公司应在本判决生效之日起十日内履行完毕；四、驳回被上诉人李某某的其他诉讼请求。 上诉人中国×公司不服上述判决，向本院提起上诉称，1、上诉人中国×公司与被上诉人李某某劳动关系于2007年2月1日解除。关于上诉人中国×公司与被上诉人李某某的劳动关系，被上诉人李某某曾于2007年2月1日提出仲裁，（2008）深福法民四重第一号民事判决对此己作出生效判决，认定双方劳动关系已于2007年2月1日解除。在2008年6月前，被上诉人李某某虽仍登记为上诉人中国×公司的法定代表人，但并不能因此得出其与上诉人中国×公司的劳动关系仍然存续。出现此种情况，是由于上诉人中国×公司没有及时变更工商登记。况且，被上诉人李某某在2007年2月1日后，便一直没有参加过上诉人中国×公司的任何工作。2、被上诉人李某某的工作年限应从2000年5月26日起算。上诉人中国×公司与中国×服务中心（深圳）企业开发公司、深圳新高特实业公司等单位均具有独立法人资格。被上诉人李某某在上述单位工作，既无人事调动关系，也无派遣关系。而且，被上诉人李某某也未能提供充分证据证明其2000年5月前在其他单位任职系上诉人中国×公司的调动或安排。所以，上诉人中国×公司、被上诉人李某某间的劳动关系和被上诉人李某某与其他公司的劳动关系之间并无延续关系，其工龄应重新计算。综上，请求二审法院判令：1、撤销（2009）深福法民四初字筅3698号民事判决书第一，二、三项判决：2、由被上诉人李某某承担本案一、二审诉讼费用。 本院经二审审理查明，原审查明事实无误，本院予以确认。 本院认为，本案双方当事人劳动关系明确，应当受劳动法律、法规保护及调整。对于上诉人中国×公司的上诉请求，本院逐一分析认定如下： 一、关于上诉人中国×公司与被上诉人李某某何时解除劳动关系的问题。深圳市福田区人民法院2009年5月31日作出（2008）深福法民四重字第1号民事判决，认为："上诉人中国×公司以被上诉人李某某不能完成公司经营指标，造成巨额经营亏损为由，免去被上诉人李某某公司董事长、副董事长、总经理职务并解除其与上诉人中国×公司之间的事实劳动关系。被上诉人李某某要求确认双方之间仍然存在劳动关系的主张，因上诉人中国×公司不同意，且双方关于继续履行事实劳动关系的具体内容及期限无法协商达成一致，如果双方事实劳动关系仍然延续，将不利于上诉人中国×公司的经营和稳定，也不利于被上诉人李某某事业的发展。因此，本院认为应当认定被上诉人李某某与上诉人中国×公司双方的劳动关系已经解除。…"，据此，深圳市福田区人民法院判决："一、被上诉人李某某与上诉人中国×公司之间的事实劳动关系已解除；……"。本院认为，深圳市福田区人民法院（2008）深福法民四重字第1号民事判决已经发生法律效力。该生效判决对于本案被上诉人李某某要求确认双方之间仍然存在劳动关系的主张已予以驳回，并判决双方劳动关系已经解除。结合被上诉人李某某从2007年2月开始即处于停工状态这一事实，本院认为，被上诉人李某某与上诉人中国×公司之间的事实劳动关系自2007年2月份已经解除。被上诉人李某某辩称根据工商登记，其在2007年2月以后仍为上诉人中国×公司的法定代表人。但根据上述认定事实，该工商登记与事实情形明显不符，属于工商登记变更迟延，而不应当作为认定事实的依据。被上诉人李某某相关答辩理由不成立，本院不予采信。上诉人中国×公司的相关上诉请求具有事实和法律依据，本院依法予以支持。 二、关于解除劳动关系的经济补偿金问题。双方当事人对于解除劳动关系前被上诉人李某某每月应发工资为人民币10120元这一事实并无异议。关于计算经济补偿金的工龄问题，本院认为，被上诉人李某某于1988年6月入职上诉人中国×公司前身中国（深圳）×企业总公司，其后又任职于中国×服务中心（深圳）企业开发公司、深圳新高特实业公司（后更名为深圳市新特高实业有限公司）。上述两公司虽然具有独立法人资格，但上诉人中国×公司均系该两公司的股东。因此应当认定上述三公司之间具有关联性，属于关联企业。被上诉人李某某系在关联公司之间的内部调动，相关工龄应当予以连续计算。因此，上诉人中国×公司应当向被上诉人李某某支付的经济补偿金为人民币192280元（10120元×19）。上诉人中国×公司未按照法律规定向被上诉人李某某支付上述经济补偿金，依据《违反和解除劳动合同的经济补偿办法》第十条的规定，还应当支付50％额外经济补偿金，即人民币96140元。 综上，上诉人中国×公司的上诉请求部分具有事实与法律依据，本院依法予以支持，其余部分缺乏法律依据，本院依法予以驳回。原审判决认定事实清楚，但适用法律存在部分不当，本院予以纠正。依据《深圳经济特区劳动合同条例》第三十条、第三十一条、《违反和解除劳动合同的经济补偿办法》第十条、《中华人民共和国民事诉讼法》第一百五十三条第一款第（二）项的规定，判决如下： 一、撤销深圳市福田区人民法院（2009）深福法民四初字第3698号民事判决第二项； 二、变更深圳市福田区人民法院（2009）深福法民四初字第3698号民事判决第一项为：确认被上诉人李某某与上诉人中国（深圳）×企业股份有限公司的事实劳动关系于2007年2月解除； 三、变更深圳市福田区人民法院（2009）深福法民四初字第3698号民事判决第三项为：上诉人中国（深圳）×企业股份有限公司应当于本判决生效之日起十五日内支付被上诉人李某某解除劳动关系的经济补偿金192280元及50％的额外经济补偿金96140元； 四、驳回中国（深圳）×企业股份有限公司的其他上诉请求。 如果未按本判决指定的期间履行金钱给付义务的，应当依照《中华人民共和国民事诉讼法》第二百二十九条的规定，加倍支付迟延履行期间的债务利息。 一、二审案件受理费人民币15元，由上诉人中国（深圳）×企业股份有限公司负担10元，由被上诉人李某某负担5元。 本判决为终审判决。 审判长彭琛 代理审判员黎奇 代理审判员肖立昕 二〇一一年五月三十一日 书记员叶云（兼） 附：相关法律法规 一、《深圳经济特区劳动合同条例》第三十条、第三十一条 第三十条因下列情形之一，解除劳动合同的，用人单位应向员工发放经济补偿金： （一）由用人单位提出，经双方协商同意解除劳动合同的； （二）用人单位依本条例第十九条第一款、第二十条第一款的规定解除劳动合同的； （三）依本条例第二十五条第（二）、（三）、（四）项的规定解除劳动合同的。 第三十一条经济补偿金的发放标准按员工在本单位的连续工作年限计算：每满一年，发给员工一个月的月工资；满半年不满一年的，按一年计发；不满半年的，发给半个月的月工资。 依本条例第十九条第一款第（二）项和依第三十条第一项解除劳动合同的，发放的经济补偿金最高不超过十二个月的月工资。 前款月工资以劳动合同解除前三个月员工的月平均工资计算。 二、《违反和解除劳动合同的经济补偿办法》第十条 用人单位解除劳动合同后，未按规定给予劳动者经济补偿的，除全额发给经济补偿金外，还须按该经济补偿金数额的百分之五十支付额外经济补偿金。 三、《中华人民共和国民事诉讼法》第一百五十三条第一款第（二）项 第二审人民法院对上诉案件，经过审理，按照下列情形，分别处理： …… （二）原判决适用法律错误的，依法改判； """
print(f"# of characters: {len(TEST_STR)}")
tokenizer = AutoTokenizer.from_pretrained(model_id)
print(f"Tokenizer: {tokenizer}")
TEST_TOK_IDS = tokenizer(TEST_STR)
print(f"# of tokens: {len(TEST_TOK_IDS['input_ids'])}")

# of characters: 7149


/ssddata/tongyx/miniconda3/envs/ise/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Tokenizer: BertTokenizerFast(name_or_path='infgrad/stella-base-zh-v3-1792d', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
# of tokens: 6692


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(model_id)
# model = SentenceTransformer("infgrad/stella-large-zh-v3-1792d")
print(model)

/ssddata/tongyx/miniconda3/envs/ise/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 1792, 'bias': True, 'activation_function': 'torch.nn.modules.linear.Identity'})
)


In [ ]:
with open(KW2IDS_FPATH, "r") as f:
    kw2doc_ids: dict[str, list[str]] = json.load(f)

ALL_IDS = []
for doc_ids in kw2doc_ids.values():
    ALL_IDS += doc_ids
# Sampling from a set deprecated since Python 3.9 and will be removed in a subsequent version.
ALL_IDS = list(set(ALL_IDS))

In [ ]:
legal_id_et_pairs = load_ets(LEGAL_DATA_DIRPATH, xml_ids=ALL_IDS)
legal_id2et = dict(legal_id_et_pairs)

Loading XMLs: 100%|██████████| 1000/1000 [00:01<00:00, 680.45it/s]

Successfully loaded 1000 ETs


In [ ]:
for legal_et in legal_id2et.values():
    root = legal_et.getroot()
    for child in root:
        print(f"### {child.tag}")
        for k, v in child.attrib.items():
            print(f"#### {k}")
            print(v)
        # if child.tag == "QW":
        #     print(child.attrib["oValue"])
    break

### QW
#### nameCN
全文
#### oValue
广东省深圳市中级人民法院 民事判决书 （2010）深中法民六终字第5494号 上诉人（原审被告）中国（深圳）×企业股份有限公司。 法定代表人高某某，董事长。 委托代理人屈某某，男。 被上诉人（原审原告）李某某，男。 委托代理人田某某，江苏维×律师事务所深圳分所律师。 上诉人中国（深圳）×企业股份有限公司（以下简称中国×公司）与被上诉人李某某拖欠工资及经济补偿金纠纷一案，中国×公司不服深圳市福田区人民法院（2009）深福法民四初字第3698号民事判决，向本院提起上诉。本院依法组成合议庭审理了本案，现已审理终结。 原审判决查明，上诉人中国×公司董事会于2000年5月26日聘任被上诉人李某某为常务副总经理，主持公司经营和管理公司全面的工作。双方未签订劳动合同。2004年2月，经上诉人中国×公司董事会决议，聘任被上诉人李某某为董事长兼总经理。2005年3月，上诉人中国×公司经改制变更为中外合资企业，并召开了改制后的第一次股东大会。被上诉人李某某被选为董事会董事，并被聘任为董事长。2006年8月，上诉人中国×公司召开2006年度第二次董事会会议，选举李健为董事长，将被上诉人李某某改聘为副董事长。 被上诉人李某某主张其工作年限自1988年6月入职中国（深圳）×企业总公司（后更名为上诉人中国×公司）。后经上诉人中国×公司调任，先后在中国×服务中心（深圳）企业开发公司、深圳新高特实业公司（后更名为深圳市新特高实业有限公司）任职。被上诉人李某某认为，其在此期间的工龄应当连续计算。被上诉人李某某提供了上诉人中国×公司工商登记资料，该资料显示1998年5月中国（深圳）×企业总公司更名为上诉人中国×公司。上诉人中国×公司系中国×服务中心（深圳）企业开发公司、深圳新特高实业有限公司股东。 2006年9月30日起，上诉人中国×公司以被上诉人李某某管理不善为由要求被上诉人李某某停止工作。2006年10月11日，上诉人中国×公司召开2006年度第三次董事会会议，以被上诉人李某某不能完成公司的经营目标，造成巨额经营亏损为由，免除了被上诉人李某某的总经理职务。2006年10月24日，上诉人中国×公司再次召开董事会会议，以被上诉人李某某不再适应担任副董事长职位为由，免除了被上诉人李某某的副董事长职务。被上诉人李某某在职期间每月应发工资为

In [ ]:
legal_id_fulltext_pairs = get_legal_id_fulltext_pairs(legal_id_et_pairs)
legal_id2fulltext = dict(legal_id_fulltext_pairs)

  0%|          | 0/68385 [00:00<?, ?it/s]

100%|██████████| 68385/68385 [00:00<00:00, 221383.95it/s]


In [ ]:
print(legal_id2fulltext[39976])

湖南省长沙市开福区人民法院 民事判决书 （2015）开民一初字第00371号 原告湖南经视文化传播有限公司，住所地湖南省长沙市开福区金鹰影视文化城主楼8楼801房。 法定代表人何瑾，该公司总经理。 委托代理人粟建权，湖南云天律师事务所律师。 被告周浩晖。 原告湖南经视文化传播有限公司（以下简称“经视文化公司”）诉被告周浩晖名誉权侵权纠纷一案，原告经视文化公司于2015年1月13日向本院提起诉讼。本院受理后，依法组成由审判员李双临担任审判长，人民陪审员黄建忠、王运珍参加的合议庭，于2015年4月20日公开开庭进行了审理。代理书记员张凡担任庭审记录。原告经视文化公司的委托代理人粟建权到庭参加诉讼。被告周浩晖经本院传票合法传唤，无正当理由拒不到庭参加诉讼。本案现已审理终结。 原告经视文化公司诉称，周浩晖在没有任何事实依据的情况下，于2014年12月26日在新浪微博发表博文，以明显带贬损意义的语言，在公开场合散布经视文化公司是于正的“金主”，“纵容”于正“无法无天”的言论；使用“打狗也得打主人”等侮辱、诽谤性的言辞对经视文化公司名誉进行损害，并且煽动其微博粉丝、小说读者对该微博进行转发、扩散，导致经视文化公司的社会评价急剧降低，给经视文化公司声誉造成极大负面影响，已构成名誉侵权。为维护经视文化公司的合法权益，故诉至法院请求：1、判令被告周浩晖立即停止对原告经视文化公司的名誉权侵害行为，删除其在网络上所有侵权内容；2、判令被告周浩晖在其博客、微博上向原告经视文化公司赔礼道歉，消除影响、恢复名誉；3、判令被告周浩晖承担本案诉讼费及鉴定、公证等费用。 原告经视文化公司为支持其诉讼请求，向本院提交如下证据： 证据一、湖南省株洲市国信公证处（2015）湘株国证内字第00011号《公证书》。证明周浩晖于2014年12月26日在新浪微博发表的博文导致经视文化公司社会评价急剧降低，侵犯了经视文化公司名誉权等事实。 证据二、株洲市公证处财务收费凭证及公证资料打印、复印费票据，证明经视文化公司为追究周浩晖名誉侵权责任所发生的取证费用共计2280元。 被告周浩晖书面辩称：一、周浩晖在微博上发表的有关言论与事实相符，措词基本恰当，不构成对经视文化公司名誉权的侵犯。经视文化公司曾购买周浩晖作品影视改编权不成的事实是客观存在的。网友留言意见属于个人判断，与周浩晖无任何关系，侧面也印证了经视文化公司及于

In [ ]:
for legal_id, fulltext in legal_id2fulltext.items():
    print(legal_id, fulltext)
    break

88391 安徽省六安市中级人民法院 刑事裁定书 （2017）皖15刑终205号 原公诉机关六安市金安区人民检察院。 上诉人（原审被告人）马永乐，男，汉族，1966年12月24日出生，安徽省霍邱县人，大专文化，原霍邱县地税局征管分局局长，副科级，住安徽省霍邱县。被告人马永乐因涉嫌犯滥用职权罪、受贿罪，于2016年1月19日经六安市金安区人民检察院决定，次日由六安市公安局金安分局对其执行刑事拘留，同年2月2日经六安市人民检察院决定不予逮捕，2016年2月3日六安市金安区人民检察院决定对其取保候审，六安市金安区人民法院于2016年11月11日决定对其继续取保候审，经六安市金安区人民法院决定，六安市公安局金安分局于2017年6月16日对其执行逮捕。现羁押于六安市看守所。 辩护人杨登贵，安徽靖淮律师事务所律师。 上诉人（原审被告人）兰敏，男，汉族，1966年1月5日出生，安徽省霍邱县人，大专文化，原霍邱县地税局稽查局执行股股长，住安徽省霍邱县。被告人兰敏因涉嫌犯滥用职权罪，于2016年3月2日被六安市金安区人民检察院取保候审，六安市金安区人民法院于2016年11月11日决定对其继续取保候审。 六安市金安区人民法院审理六安市金安区人民检察院指控原审被告人马永乐、兰敏犯滥用职权罪、受贿罪一案，于2017年5月25日作出（2016）皖1502刑初268号刑事判决。原审被告人马永乐、兰敏不服，提出上诉。本院受理后，依法组成合议庭，经阅卷，讯问上诉人，听取辩护人意见，认为事实清楚，决定不开庭审理。本案现已审理终结。 原判认定：一、被告人马永乐、兰敏滥用职权事实 2011年，在由霍邱县国税局、地税局联合进行的发票检查过程中，霍邱县地税局稽查局发现霍邱县庆发矿业有限责任公司（以下简称庆发矿业）的账面股权变动有异常情况，经霍邱县地税局安排，由该局分管副局长周某带队，并抽调该局稽查局人员刘某、兰敏，征管分局局长闫某参加，至邯邢冶金矿山管理局实地调查。后查明，安徽庆发柳编集团有限公司（以下简称庆发柳编集团）在2010年转让其在庆发矿业的股权时，对相关转让股权收入没有入账，隐瞒收入4500多万元，未缴税款1000多万元。 2011年8月，霍邱县地税局获悉庆发柳编集团再次转让了部分股权给五矿邯邢矿业有限公司。为追回未缴的税款，经霍邱县地税局和庆发柳编集团、五矿邯邢矿业有限公司三方协商，于当年8月5

In [ ]:
with open(KEY2DOC_FPATH, "r") as f:
    kw2doc_ids: dict[str, list[str]] = json.load(f)

for kw, doc_ids in kw2doc_ids.items():
    kw2doc_ids[kw] = [int(doc_id) for doc_id in doc_ids]

# print(type(kw2doc_ids))

In [ ]:
class LegalMap:

    def __init__(self, map: dict[str, list[int]], data: dict[int, Any]):
        self.map = map
        self.data = data

        # 创建下拉列表
        self.dropdown = widgets.Dropdown(
            options=map.keys(),
            value=None,
            description="Select key:",
        )

        # 输出内容
        self.output = widgets.Output()

    def on_change(self, change):
        self.output.clear_output()
        query = change["new"]
        legal_ids = self.map[query]
        markdown_str = ""
        for legal_id in legal_ids:
            markdown_str += f"""### ID: {legal_id}\n\n{self.data[legal_id]}\n\n"""

        with self.output:
            display(Markdown(markdown_str))

    def display(self):
        # 显示下拉列表和输出内容
        display(self.dropdown, self.output)

        # 绑定下拉列表的变化事件
        self.dropdown.observe(self.on_change, names="value")


legal_map = LegalMap(map=kw2doc_ids, data=legal_id2fulltext)
legal_map.display()

Dropdown(description='Select key:', options=('名誉权', '网络科技公司', '斗鱼', '法院裁定', '撤诉申请', '裁定书', '诉讼代理人', '被告公司', '受…

Output()